# Editais

Editais site licitações

In [1]:
! gdown --id 19um5yhvJVc4-RBxhG9bqQ2onxbNacrsu

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19um5yhvJVc4-RBxhG9bqQ2onxbNacrsu
To: /content/editaisDocx.rar
100% 9.03M/9.03M [00:00<00:00, 44.3MB/s]


# Install

In [ ]:
!pip install pyunpack
!pip install patool

## Jellyfish

In [ ]:
!pip install jellyfish

## Python-docx

In [ ]:
!pip install python-docx

# Extracão DOCX -> TXT e lista de sentenças

In [5]:
!mkdir editais

In [6]:
!mkdir editaistxt

In [7]:
from pyunpack import Archive
Archive('./editaisDocx.rar').extractall('./editais')

In [8]:
import os
from docx import Document

filename = os.listdir('./editais/')[1]
docPath = './editais/' + filename
doc = Document(docPath)

print(filename)
txt = ''
sentences = []

for p in doc.paragraphs:
## docs do site de licitaçõesif p.runs[0].font.color.rgb != (190, 190, 190): ## docs do site de licitações
  stripedString = p.text.strip()
  txt += stripedString + "\n"
  if(stripedString not in ["", " "]):
    sentences.append(stripedString)

  ## Checagem pelo fim do termo de referência
    endOfTermo =  all(elem in stripedString.lower()  for elem in ["anexo", "- minuta do contrato"])
    endOfTermo2 =  all(elem in stripedString.lower()  for elem in ["anexo", "- minuta de contrato"])
    endOfTermo3 =  all(elem in stripedString.lower()  for elem in ["anexo", "- minuta da ata de registro"])
    if(endOfTermo or endOfTermo2 or endOfTermo3):
      break

PREGÃO PRESENCIAL SRP 076 2022- CASA CIVIL - AQUISICAO DE MATERIAL DE CONSUMO.docx


# Código modularizado

## Funções

### Remoção de sentenças similares por Levenshtein

In [9]:
## FUNÇÃO AUXILIAR DA remove_similar_by_levenshtein()
## Identifica os índices a serem removidos e retorna um array de índices (int)

import jellyfish as jf

def get_removed_index(sentences_array):
  repeated = []

  for i in range(len(sentences_array)):
    aux = []

    #busca dentro da lista de repetidas
    if(len(repeated) > 0):
      for j in repeated:
        aux.append(jf.levenshtein_distance(sentences_array[i], sentences_array[j]))

      filtered = filter(lambda score: score <= 5, aux)

      #se é semelhante a ao menos 1 item da lista, é adicionado
      if(len(list(filtered)) > 0):
        repeated.append(i)
    else:

      #busca em todo o doc
      for s in sentences_array:
        aux.append(jf.levenshtein_distance(sentences_array[i], s))
      

      filtered = filter(lambda score: score <= 5, aux)
      
      if(len(list(filtered)) > 15):
        repeated.append(i)
  
  return repeated

In [10]:
## FUNÇÃO AUXILIAR DA remove_similar_by_levenshtein()
## Printa as sentenças similares que serão removidas

def print_removed_sentences(sentences_array, repeated_index):
  lixo = []

  for k in repeated_index:
    lixo.append(sentences_array[k])

  print("Foram removidas " + str(len(lixo)) + " sentenças")
  print("Sentenças removidas:")
  
  for sent in lixo:
    print(sent)

In [11]:
## FUNÇÃO AUXILIAR DA remove_similar_by_levenshtein()
## Remove as sentenças a partir de um array de índices, retorna um novo array

def remove_sentences_by_index(sentences_array, index_array):
  cleanSentences = []

  for i in range(len(sentences_array)):
    if(i not in index_array):
      cleanSentences.append(sentences_array[i])
  
  return cleanSentences

In [12]:
import jellyfish as jf

def remove_similar_by_levenshtein(sentences_array):
  ## identifica os índices das sentenças repetidas
  repeated_sentences_index = get_removed_index(sentences_array)

  ## Print das sentenças que serão removidas
  print_removed_sentences(sentences_array, repeated_sentences_index)

  ## Retorna um novo array (sem as repetidas)
  cleanSentences = remove_sentences_by_index(sentences_array, repeated_sentences_index)

  return cleanSentences

### Segmentação dos itens

In [13]:
## Segmenta as sentenças por seção, criando uma matriz de sentenças. Retorna um novo array

import re

def segments_sentences(sentences_array):

  segmented_sentences = []
  section = '1'
  toAdd = []
  started = False;

  #segmentação dos itens
  for s in sentences_array:
    
    #busca sentenças no padrão '1. - 1.1 - 1.1.1 ...'
    if(re.match(r'^\d+[.](\d{1,2}([.]|\s)|\s)', s)):
      started = True;
      if(s.split(".")[0] == section):
        toAdd.append(s)
      else:
        segmented_sentences.append(toAdd)
        section = s.split(".")[0]
        toAdd = []
        toAdd.append(s)
  
    #busca sentenças no padrão 'a. a- a) II. II- II) ...'
    elif(re.match(r'^([a-z]|[IVX]*)[)-.]\s', s)):
      toAdd.append(s)
    
    #busca sentenças no padrão 'a.1. a.1- a.1) ...'
    elif(re.match(r'^[a-z][.][1-9][-.)]', s)):
      toAdd.append(s)
    else:
      if(started):
        toAdd.append(s)


  segmented_sentences.append(toAdd)
  
  return segmented_sentences

In [14]:
## Corrige casos de itens descontinuados, onde a quebra de linha do PDF separou sentenças que não deveria
## Modifica o array passado como parâmetro

def fix_discontinued_itens(segmented_sentences):
 
  for sub_list in segmented_sentences:
    ## busca em sub listas (seções) com pelo menos 2 sentenças
    if(len(sub_list) > 1):
      i = 1
      while (i < len(sub_list)):
        ## se sentença não começa com indicadores ex: '1. a) III-'
        if(not re.match(r'^\d+[.](\d{1,2}([.]|\s)|\s)', sub_list[i]) and not re.match(r'^([a-z]|[IVX]+)[)-.]\s', sub_list[i]) 
        and not re.match(r'^[a-z][.][1-9][-.)]', sub_list[i])):
          ## se não termina com . ou ; pode ter mais
          if(sub_list[i].endswith('.') or sub_list[i].endswith(';')):
            sub_list[i - 1] += " " + sub_list[i]
            sub_list[i] = ''
            i += 1
          else:
            j = i;
            while(not sub_list[j].endswith('.') and not sub_list[j].endswith(';')):
              sub_list[i - 1] += " " + sub_list[j]
              sub_list[j] = ''
              if(j+1 < len(sub_list)):
                j += 1
              else:
                break
            sub_list[i - 1] += " " + sub_list[j]
            sub_list[j] = ''
            i = j + 1
        else:
          i += 1
  
  ## as sentenças corrigidas são marcadas com '' para serem removidas e não ficar duplicadas
  fixed_sentences = []

  for sub_list in segmented_sentences:
    auxL = []
    for sent in sub_list:
      if(sent != ''):
        auxL.append(sent)
    fixed_sentences.append(auxL)
  
  return fixed_sentences

In [15]:
## Corrige casos de 2 itens (sentenças) em 1 só
## Modifica o array passado como parâmetro

def fix_2_itens_same_sentence(segmented_sentences):
  for i in range(len(segmented_sentences)):
    for j in range(len(segmented_sentences[i])):
      if(re.search(r'\s\d+[.]((\d+[.])*|\d*)*\s[A-Z]', segmented_sentences[i][j])):
        index = re.search(r'\s\d+[.]((\d+[.])*|\d*)*\s[A-Z]', segmented_sentences[i][j]).span()[0]
        auxString = segmented_sentences[i][j][index + 1:]
        segmented_sentences[i][j] = segmented_sentences[i][j][0:index]
        segmented_sentences[i].insert(j + 1, auxString)

# Carregar modelo

###Carregando modelo

In [16]:
! gdown --id 19lTATTANgEH5EXSAgR8HcGQHxhKS8Olj

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19lTATTANgEH5EXSAgR8HcGQHxhKS8Olj
To: /content/model.zip
100% 1.24G/1.24G [00:10<00:00, 120MB/s]


In [17]:
!unzip "./model.zip" -d "./"

Archive:  ./model.zip
   creating: ./content/model/
  inflating: ./content/model/pytorch_model.bin  
  inflating: ./content/model/config.json  


### Installs e Imports

In [18]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 18.3 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [19]:
import os
import torch
import torch.nn as nn
from torch import optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader


### Inicializando modelo

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nclasses = 7
learning_rate = 1e-5
write_path = 'model'

In [21]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("./content/model",
                                                       num_labels=nclasses,
                                                      output_attentions = False,
                                                      output_hidden_states = False).to(device)
# model.cuda()

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Aplicação do modelo nas sentenças de um documento

## Funções auxiliares

In [22]:
## FUNÇÃO AUXILIAR DA apply_model
## Adiciona o elemento no array apropriado de acordo com sua label

def add_to_right_array(elem, label):
  if(label == 0):
    objeto.append((elem, label))
  if(label == 1):
    qualificacao_tecnica.append((elem, label))
  if(label == 2):
    prazo_pagamento.append((elem, label))
  if(label == 3):
    clausula_atraso.append((elem, label))
  if(label == 4):
    prazo_entrega.append((elem, label))
  if(label == 5):
    local.append((elem, label))

In [23]:
## FUNÇÃO AUXILIAR DA apply_model
## Extrai apenas o texto da sentenças, ignorando a numeração da seção "1.1 2.2 3.3"

def extract_only_text(txt):
  if(re.match(r'^\d+[.](\d{1,2}([.]|\s)|\s)', txt)):
      index_init = re.match(r'^\d+[.](\d{1,2}([.]|\s)|\s)', txt).span()[1]
  else:
    index_init = 0
    
  return index_init

In [24]:
## Aplica o modelo no array de sentenças segmentado, e adiciona as informações identificadas nos respectivos arrays

## Arrays para guardar as informações separadamente
objeto = []
qualificacao_tecnica = []
prazo_pagamento = []
clausula_atraso = []
prazo_entrega = []
local = []

def apply_model(segmented_sentence_array):

  for sessao in segmented_sentence_array:
    for i in range(1, len(sessao)):
      sentence = sessao[i]
      index_init = extract_only_text(sentence)
      input_text = sentence[index_init:]

      input = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
      output = model(**input)
      x = torch.argmax(output.logits, 1)
      if(x.item() < 6):
        add_to_right_array(sentence, x.item())

In [25]:
## Print das informações identificadas pelo modelo

def print_identified_info():
  print("OBJETO:")
  for el in objeto:
    print(el[0])

  print("\nQUALIFICAÇÃO TÉCNICA: \n")
  for el in qualificacao_tecnica:
    print(el[0])

  print("\nPRAZO DE PAGAMENTO: \n")
  for el in prazo_pagamento:
    print(el[0])

  print("\nCLAUSULA DE ATRASO: \n")
  for el in clausula_atraso:
    print(el[0])

  print("\nPRAZO DE ENTREGA: \n")
  for el in prazo_entrega:
    print(el[0])

  print("\nLOCAL DE ENTREGA/EXECUÇÃO DO SERVIÇO: \n")
  for el in local:
    print(el[0])

## Aplicação Fluxo completo

In [26]:
## Fluxo partindo do array de sentenças extraído do DOCX "sentences"

#Remoção de sentenças similares
clean_sentences = remove_similar_by_levenshtein(sentences)

Foram removidas 27 sentenças
Sentenças removidas:
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 3
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 4
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 5
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 6
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 7
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 8
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 12
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 13
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 14
Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008

In [27]:
# Segmentação das sentenças

segmented_sentences = segments_sentences(clean_sentences)
segmented_sentences = fix_discontinued_itens(segmented_sentences)
fix_2_itens_same_sentence(segmented_sentences)

In [28]:
segmented_sentences

[['1. DO ÓRGÃO QUE AUTORIZA A LICITAÇÃO'],
 ['2. DO OBJETO'],
 ['3. DA DOTAÇÃO ORÇAMENTÁRIA'],
 ['4. DA ATA DE REGISTRO DE PREÇOS'],
 ['5. DO LOCAL DA EXECUÇÃO DOS SERVIÇOS'],
 ['6. DAS PESSOAS QUE PODEM PARTICIPAR DA LICITAÇÃO'],
 ['7. DAS PESSOAS IMPEDIDAS DE PARTICIPAR DA LICITAÇÃO'],
 ['8. DO CREDENCIAMENTO E REPRESENTAÇÃO'],
 ['9. DA ENTREGA DOS ENVELOPES DA PROPOSTA E DA DOCUMENTAÇÃO DE HABILITAÇÃO'],
 ['10. DA PROPOSTA DE PREÇOS'],
 ['11. DO PROCEDIMENTO E JULGAMENTO DA LICITAÇÃO'],
 ['12. DA HABILITAÇÃO'],
 ['13. DA IMPUGNAÇÃO E ESCLARECIMENTOS DO ATO CONVOCATÓRIO'],
 ['14. DA FORMALIZAÇÃO DA ATA DE REGISTRO DE PREÇOS'],
 ['15. DO USUÁRIO DA ATA DE REGISTRO DE PREÇOS'],
 ['16. DOS ACRÉSCIMOS E SUPRESSÕES DA ATA DE REGISTRO DE PREÇOS Edital Pregão Presencial SRP n 076/2022-CASA CIVIL (4008415) SEI 4002.013160.00018/2022-01 / pg. 1'],
 ['17. DAS OBRIGAÇÕES DA EMPRESA CONTRATADA'],
 ['18. DO CONTROLE E DAS ALTERAÇÕES DE PREÇOS'],
 ['19. DOS RECURSOS, DA ADJUDICAÇÃO E DA HOMOLOGAÇÃ

In [29]:
# Aplicação do modelo

apply_model(segmented_sentences)

## print das informações identificadas

print_identified_info()

OBJETO:
2.1. Registro de preços para aquisição de material de consumo (materiais elétricos e eletrônicos), visando atender às eventuais necessidades da Secretaria de Estado da Casa Civil e anexos, conforme condições, quantidades e exigências estabelecidas neste Termo de Referência.
1.1. Registro de preços para aquisição de material de consumo (materiais elétricos e eletrônicos), visando atender às eventuais necessidades da Secretaria de Estado da Casa Civil e anexos, conforme condições, quantidades e exigências estabelecidas neste Termo de Referência.
4.1. A realização do certame objetiva o Registro de preços para aquisição de material de consumo materiais elétricos e eletrônicos, visando atender às eventuais necessidades da Secretaria de Estado da Casa Civil e anexos,. O objeto ora licitado têm como fonte a ser utilizada o Recurso Próprio (Fonte 100).

QUALIFICAÇÃO TÉCNICA: 

a) Atestado de capacidade técnica, expedidos por pessoas jurídicas de direito público ou privado, que comprove